# Identity-Based Multi Signatures Based on RSA
## Team:
## Pranav Vigneshwar Kumar(181CO239)
## Dhanwin Rao(181CO217)
## Arnav Santosh Nair(181CO209)

In [ ]:
# Import Statements
from rsa import key
import random
import math
import hashlib
import json

In [ ]:
# Global Variables
N, e, d, p, q = 0, 0, 0, 0, 0 # RSA values defining the public and private keys, generated by the key distribution center  
m = 0 # Message being signed by the scheme
phiOfN = 0
n = 0 # Number of signers
H2_mapping = {} #Hash map to store the function H2
L = [] # Multiset of all users' identity strings
xi = [] # Each signer's secret key for this multi-sign, generated by the key distribution center
ri = [] # Each signer's random pick from the set Zn*
Ri = [] # Each signer's RSA-encrypted pick
ti = [] # Each signer's hashed encrypted pick
si = [] # Each signer's share of the final multi-sign
multiSign = [] # Components of the final multi-sign for the current transaction

## The following section includes Helper Functions that will be utilized for the calculations of the algorithm

In [ ]:
# H0 - Maps from {0, 1}* to {0, 1}^l0
def H0(hash_input):
  hash_input = str(hash_input)
  encoding = hashlib.sha256(hash_input.encode())
  return encoding.digest()

In [ ]:
# H1 - Maps from {0, 1}* to {0, 1}^l1
def H1(hash_input):
  hash_input = str(hash_input)
  encoding = hashlib.sha1(hash_input.encode())
  return int.from_bytes(encoding.digest(), 'big')

In [ ]:
# H2 - Maps from {0, 1}* to Zn*
def H2(hash_input):
  # Return the desired hash from the mapping if it has already been calculated
  if hash_input in H2_mapping.keys():
    return H2_mapping[hash_input]


  # Pick a random index of the set Zn* to map this input to
  ZstarN_index = random.randint(1, phiOfN)
  
  # Calculate the member of Zn* this index corresponds to
  hash_output = ZstarN_index
  pCount = 0
  qCount = 0
  pNewCount = hash_output // p
  qNewCount = hash_output // q
  while (pNewCount != pCount) | (qNewCount != qCount):
    hash_output += (pNewCount - pCount) + (qNewCount - qCount)
    pCount = pNewCount
    qCount = qNewCount
    pNewCount = hash_output // p
    qNewCount = hash_output // q

  H2_mapping[hash_input] = hash_output
  return hash_output

## The following section includes the functionality to implement the Key Distribution Center.

In [ ]:
# Function to Perform RSA and generate the master public and private keys - Called by distribution center
def gen_key_pair(length_of_N): 
  (public_key, private_key) = key.newkeys(length_of_N)
  global N
  N = private_key.n
  global e
  e = private_key.e
  global d
  d = private_key.d
  global p
  p = private_key.p
  global q
  q = private_key.q

In [ ]:
# Function to calculate each user's 'xi' value - Called by distribution center
def gen_xi(ID):
  # Generate the secret key for a given signer
  xi = pow(H2(ID), d, N)
  return xi

In [ ]:
## Key Distribution Center
def KDC():
  # Perform RSA to obtain the master keys
  gen_key_pair(1024)

  # Calculate phi(N) for future use
  global phiOfN
  phiOfN = (p - 1) * (q - 1)

  # Generate each user's secret key xi, used to calculate their share of the sign
  for i in range(0, n):
    xi.append(gen_xi(L[i]))

## The following section includes the functions that each user invokes throughout the various stages of the algorithm

In [ ]:
# Function to accept the Multiset of Identity Strings and return a unique encoding 
def multisetEncoding(L):
  serialized_list = json.dumps(L)
  encoding = hashlib.sha1(serialized_list.encode())
  return int.from_bytes(encoding.digest(), 'big')  

In [ ]:
# Function to concatenate terms as input for a hash function
def inputConcat(R, encoding, m):
  R_bin = bin(R)[2:]
  encoding_bin = bin(encoding)[2:]
  m_bin = bin(m)[2:]

  finalInput = int(R_bin + encoding_bin + m_bin, 2)
  return finalInput

In [ ]:
# Function to calculate each user's 'ri', 'Ri' and 'ti' values
def round1():
  ri_signer = H2(random.randint(-100, 0))
  Ri_signer = pow(ri_signer, e, N)
  ti_signer = H0(Ri_signer)

  return [ri_signer, Ri_signer, ti_signer]

In [ ]:
# Function to validate ti and Ri pairs received by each signer
def validate_ti_Ri(ti, Ri):
  ti_expected = H0(Ri)
  return (ti == ti_expected)

In [ ]:
# Identity Based Multi Signature Scheme
## Accept the identity strings of the signers of this transaction
IDs = input("Enter the identity strings of the signers:").split(',')
global L
L = [int(i) for i in IDs]
global n
n = len(L)
global m
m = int(input("Enter the message to be signed:"))
H2_mapping.clear()

## Obtain the unique encoding for this multiset of IDs
encoding = multisetEncoding(L)
print("MultiSet Encoding for the Identity Strings(<L>): ", encoding, "\n")

## Invoke the Key Distribution Center to generate RSA keys as well as signer secret keys
KDC()
print("***Key Derivations for Master Keys and Signer Secret Keys Complete***", "\n")

## RSA-Based IBMS - Round 1
ri.clear()
Ri.clear()
ti.clear()
for i in range(0, n):
  round1_values = round1()
  ri.append(round1_values[0])
  Ri.append(round1_values[1])
  ti.append(round1_values[2])
print("Values of Randomly Selected Numbers from Zn*(ri): ", ri, "\n")
print("Values of Chosen Encrypted Randomness(Ri): ", Ri, "\n")
print("Values of Hashed Randomness(ti): ", ti, "\n")

## At the end of Round 1: All signers share their ti values with each other

## RSA-Based IBMS - Round 2: All signers share their Ri values with each other

## RSA-Based IBMS - Round 3
continueIBMS = 1
for i in range(0, n):
  continueIBMS &= validate_ti_Ri(ti[i], Ri[i])

if continueIBMS:
  # Calculate cumulative R-value
  R = 1
  for i in range(0, n):
    R = ((R % N) * (Ri[i] % N)) % N
  print("Value of Combined Randomness(R)", R, "\n")

  # Calculate sign exponent - c
  sign_exponent_input = inputConcat(R, encoding, m)
  c = H1(sign_exponent_input)
  print("Value of Sign Exponent(c): ", c, "\n")
  # Generate the sign shares of all signers
  si.clear()
  for i in range(0, n):
    si.append((ri[i] * pow(xi[i], c, N)) % N)
  print("Values of MultiSign Shares of the Signers: ", si, "\n")
else:
  print("The values shared among the signers don't match up. The algorithm has halted execution.")

## At the end of Round 3: All signers share their respective sign shares with each other

## RSA-Based IBMS - Round 4: Calculate the final multisign - s
s = 1
for i in range(0, n):
    s = ((s % N) * (si[i] % N)) % N
print("Value of Combined MultiSign Shares: ", s, "\n")

global multiSign
multiSign = [c, s]
print("The Final MultiSignature is:", multiSign, "\n")

Enter the identity strings of the signers:1,2,3,4,5,6,7
Enter the message to be signed:250
MultiSet Encoding for the Identity Strings(<L>):  576041160007194027441260930860922989323029943909 

***Key Derivations for Master Keys and Signer Secret Keys Complete*** 

Values of Randomly Selected Numbers from Zn*(ri):  [104512903556869446084752635279797541943352150662536729775991086856470468504915385445951151837585421586261078073074559275932391717303937786073290429126766924673106528934067262710826972059724047260593795451351275439113574853363655610397385702803863028979528183086688882216520075159869115901438356483256891719393, 54596212650155192508762599731268388463144191241217494169154391078836660062581268901508411894344142494595191223049030049788367264244666523924405006448002567254263440399033900565631282399019569023356350377205029986905336791767923176501171088972929374847683399317851772134390808757494048046191046095422711077833, 404655166016928229930256421848898273333351710700352195660037099

In [ ]:
## Verification 
def modInverse(b, m):
  global phiOfN
  return pow(b, phiOfN - 1, m) 

def modDivide(a, b, m): 
  a = a % m 
  inv = modInverse(b,m)  
  return ((inv % m) * (a % m)) % m

product_of_h2 = 1
for ID in L:
  product_of_h2 = ((product_of_h2 % N) * (H2(ID) % N)) % N 

# Recompute R
term1 = pow(s, e, N)
term2 = pow(product_of_h2, c, N)
R_recomp = modDivide(term1, term2, N)
print("Recomputed Value of Combined Randomness(R): ", R_recomp, "\n")

# Recompute sign exponent - c
sign_exponent_input = inputConcat(R_recomp, encoding, m)
c_recomp = H1(sign_exponent_input)
print("Recomputed Value of Sign Exponent(c): ", c_recomp, "\n")


Recomputed Value of Combined Randomness(R):  47076139682014592599850327714354560571780728481462249977049527988424408767145416714398176011499744824408777332069752705800374990755130058452605093572125327900155779689980493230368273133493391995692925433072425766840561936527929583747967125100248135879665971178520977836927799008032967232325614832611345933470 

Recomputed Value of Sign Exponent(c):  1453799137723545243642209492804892448878372194687 

